<p align="center"><b>Apple Music Annual Wrap</b></p>

<p align="center">Hanjian Xu</p>
<p align="center">May 2025</p>

---

## Goal
This notebook cleans up the AM data and prepare a final dataset for Tableau visualization.

### Data files that Apple provided:
`Apple Music Play Activity.csv` - Contains all play activity in AM from 2016 to Mar 17, 2025 (Album and Song names). Not included for PII.

`Apple Music - Track Play History` - Contains all tracks played before (Artist and Song names)

- Left Unused:
`Apple Music - Container Details.csv` - Contains Container information (Album/EP and Artist, no single track) within 2024-2025.


In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Prepare libraries/modules needed
# !pip install python-dotenv > /dev/null 2>&1
# !pip install shap > /dev/null 2>&1

import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

import warnings

# Set the style for the matploftlib library
plt.style.use('ggplot')
height = 6
aspect = 8 / height
print(os.getcwd())

/Users/xuhanjian/Desktop/Portfolio_Projects/3_AM_Wrap/project


## 1. Loading the Play Activity Data

In [4]:
df = pd.read_csv('./data/Apple Music Play Activity.csv')

# keep original df so don't need to load it everytime.
df_original = df.copy()
# Checking missing data
df.isnull().sum()

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/3072894164.py:1: DtypeWarning: Columns (6,9,14,49,59,70,95,97,108,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/Apple Music Play Activity.csv')


Age Bucket                    289995
Album Name                    128285
Apple ID Number                    0
Apple Music Subscription           0
Auto Play                      21590
                               ...  
User’s Audio Quality          262674
User’s Playback Format        262674
UTC Offset In Seconds              0
Vocal Attenuation Duration    252462
Vocal Attenuation Model ID    351816
Length: 134, dtype: int64

In [5]:
# Inspecting column names
print(df_original.columns.tolist())

['Age Bucket', 'Album Name', 'Apple ID Number', 'Apple Music Subscription', 'Auto Play', 'Build Version', 'Bundle Version', 'Camera Option', 'Carrier Name', 'Client Build Version', 'Client Device Name', 'Client IP Address', 'Client Platform', 'Container Album Name', 'Container Artist Name', 'Container Global Playlist ID', 'Container ID', 'Container iTunes Playlist ID', 'Container Library ID', 'Container Name', 'Container Origin Type', 'Container Personalized ID', 'Container Playlist ID', 'Container Radio Station ID', 'Container Radio Station Version', 'Container Season ID', 'Container Type', 'Contingency', 'Device App Name', 'Device App Version', 'Device Identifier', 'Device OS Name', 'Device OS Version', 'Device Type', 'Display Count', 'Display Language', 'Display Type', 'End Position In Milliseconds', 'End Reason Type', 'Evaluation Variant', 'Event End Timestamp', 'Event ID', 'Event Post Date Time', 'Event Reason Hint Type', 'Event Received Timestamp', 'Event Start Timestamp', 'Event

1. Clean up the column names.

In [ ]:
df.columns = (
    df.columns.str.strip()                         # remove leading/trailing whitespace
              .str.replace(r"\s+", "_", regex=True) # replace spaces with underscores
              .str.replace("'", "", regex=False)    # remove apostrophes
              .str.replace(r"[^\w_]", "", regex=True)  # remove other non-word characters
              .str.lower()
)
# df.columns[df.columns.str.contains("name", case=False)]

2. Combining `container_album_name` to `album_name` to avoid redundance.

In [11]:
df['album_name_final'] = df['album_name'].combine_first(df['container_album_name'])
df.head()

,age_bucket,album_name,apple_id_number,apple_music_subscription,auto_play,build_version,bundle_version,camera_option,carrier_name,client_build_version,...,subscription_pool_type,subscription_user_id,use_listening_history,users_audio_quality,users_playback_format,utc_offset_in_seconds,vocal_attenuation_duration,vocal_attenuation_model_id,play_ratio,album_name_final
0,NaN,Cinderella (Soundtrack from the Amazon Origina...,25057964963,True,AUTO_OFF,"Music/3.1 iOS/14.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,False,HIGH_QUALITY,SPATIAL,-14400,NaN,NaN,0.000501,Cinderella (Soundtrack from the Amazon Origina...
1,35-44,Cinderella (Soundtrack from the Amazon Origina...,25057964963,True,AUTO_OFF,"Music/3.1 iOS/17.4.1 model/iPhone14,6 hwp/t811...",3.1,NaN,NaN,NaN,...,X24,1.608221e+09,False,LOSSLESS,STEREO,-14400,0.0,NaN,-0.393715,Cinderella (Soundtrack from the Amazon Origina...
2,NaN,Cinderella (Soundtrack from the Amazon Origina...,25057964963,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/14.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,False,HIGH_QUALITY,SPATIAL,-14400,NaN,NaN,0.000000,Cinderella (Soundtrack from the Amazon Origina...
3,NaN,Cinderella (Soundtrack from the Amazon Origina...,25057964963,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/15.3.1 model/iPhone12,8 hwp/t803...",3.1,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,-18000,NaN,NaN,-0.428484,Cinderella (Soundtrack from the Amazon Origina...
4,NaN,Cinderella (Soundtrack from the Amazon Origina...,25057964963,True,AUTO_OFF,"Music/3.1 iOS/14.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,False,HIGH_QUALITY,SPATIAL,-14400,NaN,NaN,0.007210,Cinderella (Soundtrack from the Amazon Origina...


3. Selecting columns to keep

In [12]:
columns_to_keep = [
    'song_name', 'album_name_final', 'media_type', 'event_type', 'feature_name',
    'event_start_timestamp', 'play_duration_milliseconds', 'end_reason_type',
    'media_duration_in_milliseconds', 'users_audio_quality', 'users_playback_format',
    'ip_city', 'ip_country_code', 'ip_region_code'
]

df = df[columns_to_keep]
# only select audio playing
df = df[(df['media_type'] == 'AUDIO') & (df['song_name'] != 'na')]
df.rename(columns={'album_name_final': 'album_name'}, inplace=True)
df.head()

,song_name,album_name,media_type,event_type,feature_name,event_start_timestamp,play_duration_milliseconds,end_reason_type,media_duration_in_milliseconds,users_audio_quality,users_playback_format,ip_city,ip_country_code,ip_region_code
0,Million To One / Could Have Been Me (Reprise),Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2021-09-15T12:04:03.722Z,77926.0,NATURAL_END_OF_TRACK,77887.0,HIGH_QUALITY,SPATIAL,NaN,NaN,NaN
1,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,Siri-actions-local,2024-05-12T02:13:25.145Z,168506.0,PLAYBACK_SUSPENDED,277932.0,LOSSLESS,STEREO,TORONTO,CA,ON
2,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,search / album_detail,2021-09-04T21:10:06.670Z,277932.0,NATURAL_END_OF_TRACK,277932.0,HIGH_QUALITY,SPATIAL,NaN,NaN,NaN
3,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2022-03-03T21:01:42.785Z,79703.0,SCRUB_BEGIN,139459.0,NaN,NaN,NaN,NaN,NaN
4,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2021-09-17T18:27:24.788Z,140394.0,NATURAL_END_OF_TRACK,139389.0,HIGH_QUALITY,SPATIAL,NaN,NaN,NaN


4. Additional cleaning.
- Remove very short play records (duration <= 30 s is considered "invalid" play of a song)
- Convert the start of the play time to y/m/d/wkd/h information for further analysis.
- Filter out entries without song names.

In [14]:
# Remove very short plays (e.g., under 30 seconds = 30,000 ms)
df = df[df['play_duration_milliseconds'] >= 30000]

# Convert timestamp to datetime
df['event_start_timestamp'] = pd.to_datetime(df['event_start_timestamp'], errors='coerce', utc=True)

# Add derived time fields
df['date'] = df['event_start_timestamp'].dt.date
df['year'] = df['event_start_timestamp'].dt.year
df['month'] = df['event_start_timestamp'].dt.to_period('M')
df['weekday'] = df['event_start_timestamp'].dt.day_name()
df['hour'] = df['event_start_timestamp'].dt.hour

# Convert duration to minutes
df['play_duration_min'] = df['play_duration_milliseconds'] / 60000

# filter out entries without a song name
df_clean = df[~df['song_name'].isna()]
# df_clean.to_csv('data/am_play_activity_final.csv', index = False)

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/209036199.py:10: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['event_start_timestamp'].dt.to_period('M')


In [ ]:
df_clean['play_count'] = df_clean['play_duration_milliseconds']/df_clean['media_duration_in_milliseconds']

# There are 10 plays (Fall in Love at Christmas Time - Single version) without media_duration_in_milliseconds. Manually set to its play time from other entries of the same song.
df_clean[df_clean['play_count'].isna()]['media_duration_in_milliseconds'] = 218383.0

# def get_play_count(play_duration_ms, media_duration_ms):
#    MIN_PLAY_MS = 30_000  # 30 seconds in milliseconds

    # If total play duration is less than 30 seconds, don't count it
#    if play_duration_ms < MIN_PLAY_MS:
#        return 0

    # Calculate how many full plays fit into the play duration
#    full_plays = play_duration_ms // media_duration_ms

    # For the remainder: does it exceed 30 seconds? Count as 1 more if yes
#    remaining = play_duration_ms % media_duration_ms
#    if remaining >= MIN_PLAY_MS:
#        return full_plays + 1
#    else:
#        return full_plays

def get_play_count(play_duration_ms, media_duration_ms):
    MIN_PLAY_MS = 30_000

    # 1) Too short to count
    if play_duration_ms < MIN_PLAY_MS:
        return 0

    # 2) Invalid media duration → at least one play
    if media_duration_ms <= 0 or (isinstance(media_duration_ms, float) and math.isnan(media_duration_ms)):
        return 1

    # 3) Compute full plays + leftover
    full_plays, remaining = divmod(play_duration_ms, media_duration_ms)

    # 4) If leftover ≥ 30 s, count one more
    return int(full_plays) + int(remaining >= MIN_PLAY_MS)

df['play_count'] = df.apply(
    lambda row: get_play_count(row['play_duration'], row['media_duration']),
    axis=1
)

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/2556167072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['play_count'] = df_clean['play_duration_milliseconds']/df_clean['media_duration_in_milliseconds']
/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/2556167072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[df_clean['play_count'].isna()]['media_duration_in_milliseconds'] = 218383.0


5. A quick look at top played songs per year.
* Play count is calculated as counting entries (i.e., rows) for each song: In most cases (except one), each entry's play time is or less than the song duration.

In [ ]:
# a quick look at top songs 
top_songs_by_year = (
    df_clean.groupby(['year', 'song_name'], as_index=False)
      .agg(
          play_count=('song_name', 'size'),               # count plays
          total_duration_min=('play_duration_min', 'sum') # sum durations
      )
      .assign(rank=lambda x: x.groupby('year')['play_count'].rank(method='first', ascending=False))
      .query('rank <= 10')
      .sort_values(['year', 'rank'])
)
top_songs_by_year.query('year == 2025')

,year,song_name,play_count,total_duration_min,rank
13811,2025.0,我要的幸福,533,1681.127233,1.0
13706,2025.0,To You,383,1516.805400,2.0
13285,2025.0,BIRDS OF A FEATHER,291,925.700117,3.0
13439,2025.0,I Always Wanted A Brother,222,768.566483,4.0
13488,2025.0,Let's Go out!,214,470.313083,5.0
13733,2025.0,Way Forward,195,674.294783,6.0
13765,2025.0,don't wanna break up again,177,488.895300,7.0
13403,2025.0,Freedom,175,889.672700,8.0
13760,2025.0,You're Nothing but a Puppet,169,389.450333,9.0
13726,2025.0,Vote I,164,430.366800,10.0


## 2. Loading the Track History Data.

Use `rapidfuzz` to link `Apple Music - Track Play History.csv` with cleaned play history `am_play_activity_final`.

The goal is to do a probable match of song names so that each song can be single identifiable.


In [ ]:
from rapidfuzz import process, fuzz
import re

# Load track_df
track_df = pd.read_csv("data/Apple Music - Track Play History.csv")

# Standandize column names (lower case, replace space with _)
track_df.columns = track_df.columns.str.strip().str.lower().str.replace(r"\s+", "_", regex=True)
track_df.head()

,track_name,last_played_date,is_user_initiated
0,許文友 - 花房姑娘,1708608852879,False
1,許文友 - 花房姑娘,1708608852879,True
2,Skrillex & Diplo - Where Are Ü Now (with Justi...,1609690494734,False
3,Skrillex & Diplo - Where Are Ü Now (with Justi...,1609690494734,True
4,Heitor Pereira - Minions Through Time,1734025600726,False


In [ ]:
# Clean function to normalize song names
def clean_title(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"\(.*?\)", "", text)  # remove text in parentheses
    text = re.sub(r"\[.*?\]", "", text)  # remove text in brackets
    text = re.sub(r"[^\w\s]", "", text, flags=re.UNICODE)  # remove other special characters
    text = re.sub(r"\s+", " ", text).strip()  # normalize spaces
    
    return text
# Apply cleaning
df_clean["clean_song_name"] = df_clean["song_name"].apply(clean_title)

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/1120400371.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["clean_song_name"] = df_clean["song_name"].apply(clean_title)


1. Break `track_name` into two parts by " - ": `artist_name` + `track_name_clean`

- The latter column is used to match `song_name` in df_clean.
- Remove duplicated entries in `track_df` (many songs have two entries with `is_user_initiated` being `True` and `False`)

In [ ]:
# Split track_name and container_df's 
track_df[['artist_name', 'track_name_clean']] = (
    track_df['track_name']
    .str.rsplit(' - ', n=1, expand=True)
)

# Remove repeated song name entrance
track_df = track_df.sort_values(
    by='is_user_initiated',
    ascending=False
)
track_df = track_df.drop_duplicates(
    subset=['track_name_clean', 'artist_name'],
    keep='first'
)


track_df.head()

2. Matching song names across 2 datasets.

This is challenging doe to the lack of a consistent one-to-one relationship among song, album, and artist in any AM exported data. We began with a rapid fuzzy match on song titles alone. For cases with songs sharing the same titles (e.g., entirely different songs, the same song but different versions/mixes/covers), a secondary fuzzy match (partial_token_set_ratio) using album_name (from main df) and artist_name (from track_df) was done, to utilize text patterns like movie title or cast information embedded in both fields. In either case, the row would be flagged if there's a tie on that album ~ artist fuzzy match or the song_name match yielded a low score.

Flagged matches and spotchecks were manually done by filtering on artists or albums to maximize accuracy. That said, there might still be potential mismatches in the data.

In [ ]:
def is_fuzzy_match(query, choices, threshold=85):
    """
    Returns (best_match, best_score, best_idx) if either token_set or token_sort
    ratio >= threshold; else (None, 0, None).
    """
    set_match  = process.extractOne(query, choices, scorer=fuzz.token_set_ratio)
    sort_match = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)

    # unpack, guard against None
    name_set,  score_set,  idx_set  = set_match  or ("", 0, None)
    name_sort, score_sort, idx_sort = sort_match or ("", 0, None)

    # pick the better of the two
    if score_set >= score_sort:
        best_name, best_score, best_idx = name_set,  score_set,  idx_set
    else:
        best_name, best_score, best_idx = name_sort, score_sort, idx_sort

    if best_score >= threshold:
        return best_name, best_score, best_idx
    return None, 0, None

In [ ]:
results = []
for _, row in df_clean.iterrows():
    song       = row['song_name']
    album      = row['album_name']
    orig_song  = row['song_name']
    orig_album = row['album_name']

    # 1) Get your top-5 song‐title matches
    song_matches = process.extract(
        orig_song,
        track_df['track_name_clean'],
        scorer=fuzz.token_sort_ratio,
        limit=5
    )
    # find the highest song score
    max_score = max(score for _, score, _ in song_matches)
    # narrow to only those at the top
    top_matches = [m for m in song_matches if m[1] == max_score]

    # If there's a unique perfect match at 100
    if max_score == 100 and len(top_matches) == 1:
        _, song_score, idx = top_matches[0]
        tr = track_df.iloc[idx]
        results.append({
            'original_song':      orig_song,
            'original_album':     orig_album,
            'matched_track_name': tr['track_name'],
            'matched_artist':     tr['artist_name'],
            'song_match_score':   song_score,
            'album_artist_score': None,
            'flagged':            False
        })
        continue

    # Or if there're multiple ties (or scores < 100) -> pick by “album ~ artist” strength
    best = None
    best_a2s = -1
    # compute album ~ artist score for each tied candidate
    for _, song_score, idx in top_matches:
        tr = track_df.iloc[idx]
        # use a partial token‐set so that: e.g., "cinderella" vs long artist strings still hits high
        a2s_score = fuzz.partial_token_set_ratio(album, tr['artist_name'])
        if a2s_score > best_a2s:
            best_a2s = a2s_score
            best     = (tr, song_score, best_a2s)

    # detect if there’s still a tie on that album ~ artist score
    tie_count = sum(
        1
        for _, _, idx in top_matches
        if fuzz.partial_token_set_ratio(
               album,
               track_df.iloc[idx]['artist_name']
           ) == best_a2s
    )
    tr, song_score, album_artist_score = best

    results.append({
        'original_song':      orig_song,
        'original_album':     orig_album,
        'matched_track_name': tr['track_name'],
        'matched_artist':     tr['artist_name'],
        'song_match_score':   song_score,
        'album_artist_score': album_artist_score,
        'flagged':            (tie_count > 1)
    })

# convert results to df
results_df = pd.DataFrame(results)

# see how many got flagged
num_flagged = results_df['flagged'].sum()
print(f"{num_flagged} entries need manual review (tied candidates).")

2382 entries need manual review (tied candidates).


In [ ]:
# save the matching_df.csv to be manually checked.
results_df.to_csv('matching_df.csv', encoding="utf-8-sig")

After this manual review, the data was saved in `final_manual_reviewed_df.csv`

In [ ]:
# Load back the manually reviewed df, and then attach it to the df_clean to obtain the artist information. 
manual_df = pd.read_csv('data/final_manual_reviewed_df.csv', encoding="utf-8-sig")
combined_df = pd.concat([df_clean.reset_index(drop=True), manual_df.iloc[:,1:].reset_index(drop=True)], axis=1)
combined_df.head()

,Unnamed: 0,original_song,original_album,matched_track_name,matched_artist,song_match_score,album_artist_score,flagged
0,0,Million To One / Could Have Been Me (Reprise),Cinderella (Soundtrack from the Amazon Origina...,Camila Cabello & Nicholas Galitzine - Million ...,Camila Cabello & Nicholas Galitzine,100.0,NaN,False
1,1,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,"Camila Cabello, Idina Menzel & Cinderella Orig...","Camila Cabello, Idina Menzel & Cinderella Orig...",100.0,NaN,False
2,2,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,"Camila Cabello, Idina Menzel & Cinderella Orig...","Camila Cabello, Idina Menzel & Cinderella Orig...",100.0,NaN,False
3,3,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,Idina Menzel & Cinderella Original Motion Pict...,Idina Menzel & Cinderella Original Motion Pict...,100.0,100.0,False
4,4,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,Idina Menzel & Cinderella Original Motion Pict...,Idina Menzel & Cinderella Original Motion Pict...,100.0,100.0,False


3. Further cleaning (removing potential playlist playing without meaningful `song_name`)

In [ ]:
# Check some Apple music 1 and number entries (song names contains "Apple Music 1" or a string of numeric values, i.e. playlist IDs)
combined_df[combined_df['song_name'].str.contains('Apple Music 1')]
combined_df[combined_df['song_name'].str.contains(r'\d{8,}', na=False)]

# Remove such entries
combined_df = combined_df[
    ~combined_df['song_name'].str.contains('Apple Music 1|\\d{8,}', na=False)
]

# Drop redundant columns 
# df_final.drop(['original_song', 'original_album', 'song_match_score', 'album_artist_score',	'flagged'], axis = 1, inplace=True)

4. Two final reforms.

(1) Double check Play Count to see if each entry correspond to roughly 1 play:

In [ ]:
# Double check on the play count:
def get_play_count(play_duration_ms, media_duration_ms):
    MIN_PLAY_MS = 30000
    # If too short to count -> 0 (shouldn't have this situation, because this is pre-removed)
    if play_duration_ms < MIN_PLAY_MS:
        return 0
    # If media_duration_ms is 0 or NaN -> 1 play
    if media_duration_ms == 0 or (pd.isna(media_duration_ms)):
        return 1
    # Compute full plays + leftover
    full_plays, remaining = divmod(play_duration_ms, media_duration_ms)
    # If leftover >= 30 s, count one more
    return int(full_plays) + int(remaining >= MIN_PLAY_MS)
df_test = combined_df.copy()
df_test['play_count_new'] = df_test.apply(
    lambda row: get_play_count(row['play_duration_milliseconds'], row['media_duration_in_milliseconds']),
    axis=1
)

104700    0.0
140316    0.0
70150     0.0
70221     0.0
44545     0.0
         ... 
58194     NaN
85925     NaN
123227    NaN
151095    NaN
152539    NaN
Name: media_duration_in_milliseconds, Length: 164013, dtype: float64

In [ ]:
counts = df_test['play_count_new'].value_counts()
counts = counts.sort_index()
counts

play_count_new
1     164012
13         1
Name: count, dtype: int64

There's only one instance having an overly longer play_count_new (which could be a system mistake, because most of the others suggest one-play one-entry).

Thus we decided to stick to counting row # as play count.

(2) Artist column reform.

- Remove unnecessary characeters / words for featuring artists: &, and, feat., featuring.
- A strictly comma-delimited string with consistent spacing.
- For stats purposes, remove additional punctuation before suffixes like "sr" or "jr." (so that it won't be treated as a separate name).


In [ ]:
# defined functions for cleaning the name list:
def normalize_delimiters(artist_str):
    if pd.isna(artist_str): return ""
    
    # Remove comma before suffixes like "jr.", "Jr", "Sr.", etc.
    suffixes = r"\b(?:jr\.?|sr\.?|ii|iii|iv)\b"
    text = re.sub(r",\s*(?=" + suffixes + ")", " ", artist_str, flags=re.IGNORECASE)
    
    # Normalize delimiters to be ", " only
    # Replace " & "
    text = re.sub(r"\s+\&\s+", ", ", text)

    # Replace " and "
    text = re.sub(r"\s+and\s+", ", ", text, flags=re.IGNORECASE)

    # Replace " feat." or "featuring"
    text = re.sub(r"\s+(feat\.?|featuring)\s+", ", ", text, flags=re.IGNORECASE)
    
    # Finally, collapse extra commas/spaces
    tokens = [t.strip() for t in text.split(",") if t.strip()]
    return ", ".join(tokens)

# form each cell of artists as a list structure.
def smart_split_artists(artist_str):
    if pd.isna(artist_str): return []
    return [a.strip() for a in artist_str.split(",") if a.strip()]

In [168]:
combined_df['artist_name_clean'] = combined_df['matched_artist'].apply(normalize_delimiters)
combined_df['artist_list'] = combined_df['artist_name_clean'].apply(smart_split_artists)

In [ ]:
# Settings to display all rows/columns/contents
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.max_colwidth', None)    # Show full content in cells
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

# Reset the customization above
pd.reset_option("all")

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/2844998638.py:10: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/2844998638.py:10: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


Finally, save the generated df for Tableau `final_am_dataset.csv`

In [ ]:
combined_df.to_csv('data/final_am_dataset_tableau.csv', encoding="utf-8-sig")

# For sample use
# combined_df.head(1000).to_csv('data/final_am_dataset_1000.csv', encoding="utf-8-sig")

Post-hoc Fixes:
1. Several columns missing play timestamp:

- Solution: going back to the original `Apple Music Play Activity.csv` and retrieve the same song's `Event Received Timestamp` information and fill it in.

In [270]:
for idx, row in combined_df[combined_df['year'].isna()].iterrows():
    song_name = row['song_name']
    
    # Try to find the first matching row in df_wo_ts
    match = df_original[df_original['Song Name'] == song_name]
    
    if not match.empty:
        timestamp = match.iloc[0]['Event Received Timestamp']
        # Overwrite in combined_df
        combined_df.at[idx, 'event_start_timestamp'] = timestamp
        combined_df.at[idx, 'date'] = timestamp.date
        combined_df.at[idx, 'year'] = timestamp.year
        combined_df.at[idx, 'month'] = timestamp.to_period('M')
        combined_df.at[idx, 'weekday'] = timestamp.day_name()
        combined_df.at[idx, 'hour'] = timestamp.hour

/var/folders/tr/xh05sl9s3q171cth46cr9vnw0000gn/T/ipykernel_22526/1107636054.py:13: UserWarning: Converting to Period representation will drop timezone information.
  combined_df.at[idx, 'month'] = timestamp.to_period('M')


In [ ]:
# Save combined_df again. 
combined_df.to_csv('data/final_am_dataset_tableau.csv', encoding="utf-8-sig")
# combined_df.head(1000).to_csv('./data/final_am_dataset_1000.csv', encoding="utf-8-sig")

One final edit:

🎧 Categorizing Songs/Albums as Soundtrack vs. Non-Soundtrack

Considering my listening habits, I decided to categorize the song/albums altogether into soundtracks (e.g., OSTs, film scores) and regular music.

A new column `category` is created with binary labels: either "Soundtrack" or "Non-Soundtrack".

This is based on whether the album_name OR song_name contains common soundtrack-related keywords such as:
- "Soundtrack"
- "OST"
- "Original Motion Picture"
- "Motion Picture"
- "Original Series"
- "Score"
- "Movie"

I’ll use str.contains() with a case-insensitive search.

In [352]:
soundtrack_keywords = r"\bSoundtrack\b|\bOST\b|\bOriginal Motion Picture\b|\bScore\b|\bMovie\b|\bMotion Picture\b|\bOriginal Series\b|原聲|\bTelevision Series\b"

combined_df['category'] = combined_df.apply(
    lambda row: 'Soundtrack' if (
        pd.notna(row['album_name']) and pd.Series(row['album_name']).str.contains(soundtrack_keywords, case=False, regex=True, na=False).any()
    ) or (
        pd.notna(row['song_name']) and pd.Series(row['song_name']).str.contains(soundtrack_keywords, case=False, regex=True, na=False).any()
    ) else 'Non-Soundtrack',
    axis=1
)

In [ ]:
combined_df

# Save combined_df final version for tableau.
combined_df.to_csv('./data/final_am_dataset_tableau.csv', encoding="utf-8-sig")
#combined_df.head(1000).to_csv('./data/final_am_dataset_1000_tableau.csv', encoding="utf-8-sig")


,song_name,album_name,media_type,event_type,feature_name,event_start_timestamp,play_duration_milliseconds,end_reason_type,media_duration_in_milliseconds,users_audio_quality,...,play_count,clean_song_name,matched_track_name,matched_artist,song_match_score,album_artist_score,flagged,artist_name_clean,artist_list,category
0,Million To One / Could Have Been Me (Reprise),Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2021-09-15 12:04:03.722000+00:00,77926.0,NATURAL_END_OF_TRACK,77887.0,HIGH_QUALITY,...,1.000501,Million To One Could Have Been Me,Camila Cabello & Nicholas Galitzine - Million ...,Camila Cabello & Nicholas Galitzine,100.0,NaN,False,"Camila Cabello, Nicholas Galitzine","[Camila Cabello, Nicholas Galitzine]",Soundtrack
1,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,Siri-actions-local,2024-05-12 02:13:25.145000+00:00,168506.0,PLAYBACK_SUSPENDED,277932.0,LOSSLESS,...,0.606285,Rhythm Nation You Gotta Be,"Camila Cabello, Idina Menzel & Cinderella Orig...","Camila Cabello, Idina Menzel & Cinderella Orig...",100.0,NaN,False,"Camila Cabello, Idina Menzel, Cinderella Origi...","[Camila Cabello, Idina Menzel, Cinderella Orig...",Soundtrack
2,Rhythm Nation / You Gotta Be,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,search / album_detail,2021-09-04 21:10:06.670000+00:00,277932.0,NATURAL_END_OF_TRACK,277932.0,HIGH_QUALITY,...,1.000000,Rhythm Nation You Gotta Be,"Camila Cabello, Idina Menzel & Cinderella Orig...","Camila Cabello, Idina Menzel & Cinderella Orig...",100.0,NaN,False,"Camila Cabello, Idina Menzel, Cinderella Origi...","[Camila Cabello, Idina Menzel, Cinderella Orig...",Soundtrack
3,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2022-03-03 21:01:42.785000+00:00,79703.0,SCRUB_BEGIN,139459.0,NaN,...,0.571516,Material Girl,Idina Menzel & Cinderella Original Motion Pict...,Idina Menzel & Cinderella Original Motion Pict...,100.0,100.000000,False,"Idina Menzel, Cinderella Original Motion Pictu...","[Idina Menzel, Cinderella Original Motion Pict...",Soundtrack
4,Material Girl,Cinderella (Soundtrack from the Amazon Origina...,AUDIO,PLAY_END,library / album_detail,2021-09-17 18:27:24.788000+00:00,140394.0,NATURAL_END_OF_TRACK,139389.0,HIGH_QUALITY,...,1.007210,Material Girl,Idina Menzel & Cinderella Original Motion Pict...,Idina Menzel & Cinderella Original Motion Pict...,100.0,100.000000,False,"Idina Menzel, Cinderella Original Motion Pictu...","[Idina Menzel, Cinderella Original Motion Pict...",Soundtrack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164033,Hollywood and Vines,Lost: Season 1 (Original Television Soundtrack),AUDIO,PLAY_END,library / album_detail,2023-08-09 19:38:10.166000+00:00,37578.0,PLAYBACK_SUSPENDED,114056.0,NaN,...,0.329470,Hollywood and Vines,"Michael Giacchino, Tim Simonec & Hollywood Stu...","Michael Giacchino, Tim Simonec & Hollywood Stu...",100.0,NaN,False,"Michael Giacchino, Tim Simonec, Hollywood Stud...","[Michael Giacchino, Tim Simonec, Hollywood Stu...",Soundtrack
164034,Proper Motivation,Lost: Season 1 (Original Television Soundtrack),AUDIO,PLAY_END,library / album_detail,2023-07-27 19:02:22.283000+00:00,121556.0,NATURAL_END_OF_TRACK,121556.0,NaN,...,1.000000,Proper Motivation,"Michael Giacchino, Tim Simonec & Hollywood Stu...","Michael Giacchino, Tim Simonec & Hollywood Stu...",100.0,NaN,False,"Michael Giacchino, Tim Simonec, Hollywood Stud...","[Michael Giacchino, Tim Simonec, Hollywood Stu...",Soundtrack
164035,You Remember,A Week At the End of Winter - EP,AUDIO,PLAY_END,library / playlist_detail,2023-03-10 20:06:50.377000+00:00,129288.0,NATURAL_END_OF_TRACK,129288.0,NaN,...,1.000000,You Remember,cxlt. & squeeda - You Remember,cxlt. & squeeda,100.0,NaN,False,"cxlt., squeeda","[cxlt., squeeda]",Non-Soundtrack
164036,Adorare,Adorare - Single,AUDIO,PLAY_END,library / playlist_detail,2023-09-18

Some final manual adjustments were done on Hans Zimmer and some Soundtrack tracks.

Several Hans Zimmer albums were categorized as non-soundtrack because of lack of keywords in the albums.

The matching also is inaccurate in linking:
1. Janet (album) to Hans Zimmer.
2. better off (filous remix) - Single
3. Glass Onion: A Knives Out Mystery
4. Love On the Inside
5. little woman
6. keep the faith